# Astradigital Engine Asset Organization System

This notebook provides an overview and implementation details for the asset organization system used in the Astradigital Engine. It includes code samples, optimization techniques, and best practices for managing assets across WordPress, HuggingFace, and Docker deployments.

## System Architecture

The asset organization system consists of the following components:

1. **Directory Structure** - Hierarchical organization of source, optimized, and deployment assets
2. **PowerShell Scripts** - For setup, migration, and management
3. **Python Scripts** - For asset optimization and processing
4. **Documentation** - Guides and examples for system usage

The system is designed to maintain a single source of truth for all assets while providing optimized versions for different deployment targets.

## Directory Structure

```
assets/
├── source/            # Original, high-quality source files
│   ├── images/        # Original images in full resolution
│   ├── audio/         # Original audio files
│   ├── video/         # Original video files
│   └── documents/     # Original text documents and scripts
├── optimized/         # Optimized assets ready for different platforms
│   ├── wordpress/     # WordPress-specific optimizations
│   ├── huggingface/   # HuggingFace-optimized assets
│   └── web/           # General web-optimized assets
├── deployment/        # Platform-specific deployment assets
│   ├── wordpress/     # Ready for WordPress upload
│   ├── docker/        # Assets to include in Docker builds
│   └── huggingface/   # Assets ready for HuggingFace deployment
└── scripts/           # Asset processing and optimization scripts
```

## Image Optimization Libraries

The asset system uses Python's Pillow library for image processing, along with other specialized tools for format conversion and optimization:

First, let's import the necessary libraries and set up our environment:

In [ ]:
# Standard libraries
import os
import sys
import shutil
from pathlib import Path
import json
from datetime import datetime

# Image processing
try:
    from PIL import Image, ImageOps
    print("✅ Pillow is installed and imported successfully")
except ImportError:
    print("❌ Pillow is not installed. Install with: pip install pillow")
    
# Define base paths
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
ASSETS_DIR = os.path.join(BASE_DIR, "assets")

print(f"Base directory: {BASE_DIR}")
print(f"Assets directory: {ASSETS_DIR}")

## Image Optimization for Different Platforms

Each platform has specific requirements for image optimization. Let's define the optimization parameters for WordPress, HuggingFace, and Docker deployments:

In [ ]:
# Platform-specific optimization parameters
platform_configs = {
    "wordpress": {
        "featured": {"max_width": 1200, "max_height": 630, "quality": 90, "format": "JPEG"},
        "thumbnail": {"max_width": 150, "max_height": 150, "quality": 80, "format": "JPEG"},
        "content": {"max_width": 800, "max_height": None, "quality": 85, "format": "JPEG"},
    },
    "huggingface": {
        "ui": {"max_width": 600, "max_height": None, "quality": 80, "format": "WEBP"},
        "images": {"max_width": 800, "max_height": None, "quality": 75, "format": "WEBP"},
        "icons": {"max_width": 128, "max_height": 128, "quality": 90, "format": "PNG"},
    },
    "web": {
        "general": {"max_width": 1200, "max_height": None, "quality": 85, "format": "WEBP"},
        "thumbnail": {"max_width": 300, "max_height": 300, "quality": 80, "format": "WEBP"},
    }
}

print("WordPress featured image config:", platform_configs["wordpress"]["featured"])
print("HuggingFace UI image config:", platform_configs["huggingface"]["ui"])

## Image Processing Functions

Now, let's define the core image processing functions that will be used by our optimization scripts:

In [ ]:
def resize_image(image_path, output_path, config):
    """Resize and optimize an image based on configuration parameters"""
    try:
        # Open the image
        with Image.open(image_path) as img:
            # Convert to RGB if needed (for JPEG output)
            if config["format"] == "JPEG" and img.mode != "RGB":
                img = img.convert("RGB")
                
            # Calculate new dimensions while maintaining aspect ratio
            width, height = img.size
            max_width, max_height = config["max_width"], config["max_height"]
            
            # Resize logic
            if max_width and max_height:  # Both dimensions specified
                img = ImageOps.fit(img, (max_width, max_height), Image.LANCZOS)
            elif max_width and width > max_width:  # Only width constrained
                ratio = max_width / width
                new_height = int(height * ratio)
                img = img.resize((max_width, new_height), Image.LANCZOS)
            elif max_height and height > max_height:  # Only height constrained
                ratio = max_height / height
                new_width = int(width * ratio)
                img = img.resize((new_width, max_height), Image.LANCZOS)
                
            # Ensure output directory exists
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
                
            # Save with appropriate format and quality
            if config["format"] == "JPEG":
                img.save(output_path, "JPEG", quality=config["quality"], optimize=True)
            elif config["format"] == "PNG":
                img.save(output_path, "PNG", optimize=True)
            elif config["format"] == "WEBP":
                img.save(output_path, "WEBP", quality=config["quality"])
            else:
                img.save(output_path)
                
            return True, f"Optimized image saved to {output_path}"
    except Exception as e:
        return False, f"Error processing {image_path}: {str(e)}"

# Example usage
test_image = os.path.join(ASSETS_DIR, "source", "images", "sample.jpg")
if os.path.exists(test_image):
    output_path = os.path.join(ASSETS_DIR, "optimized", "wordpress", "featured", "sample-optimized.jpg")
    success, message = resize_image(test_image, output_path, platform_configs["wordpress"]["featured"])
    print(message)
else:
    print(f"Test image not found at {test_image}. This is just a demonstration of the function.")

## WordPress Asset Preparation

Here's how we prepare assets specifically for WordPress posts:

In [ ]:
def prepare_wordpress_assets(source_dir, post_title):
    """Prepare assets for a WordPress post"""
    post_slug = post_title.lower().replace(' ', '-')
    timestamp = datetime.now().strftime("%Y%m%d")
    post_dir = f"{timestamp}-{post_slug}"
    
    # Define output directories
    featured_dir = os.path.join(ASSETS_DIR, "optimized", "wordpress", "featured", post_dir)
    thumbnail_dir = os.path.join(ASSETS_DIR, "optimized", "wordpress", "thumbnails", post_dir)
    content_dir = os.path.join(ASSETS_DIR, "optimized", "wordpress", "content", post_dir)
    
    # Create directories
    os.makedirs(featured_dir, exist_ok=True)
    os.makedirs(thumbnail_dir, exist_ok=True)
    os.makedirs(content_dir, exist_ok=True)
    
    # Process images
    image_count = 0
    source_path = Path(source_dir)
    for img_file in source_path.glob("**/*.{jpg,jpeg,png,gif}"):
        # File paths
        rel_path = img_file.relative_to(source_path)
        
        # Process for different uses
        featured_output = os.path.join(featured_dir, rel_path.name)
        thumbnail_output = os.path.join(thumbnail_dir, rel_path.name)
        content_output = os.path.join(content_dir, rel_path.name)
        
        # Optimize for each use case
        resize_image(str(img_file), featured_output, platform_configs["wordpress"]["featured"])
        resize_image(str(img_file), thumbnail_output, platform_configs["wordpress"]["thumbnail"])
        resize_image(str(img_file), content_output, platform_configs["wordpress"]["content"])
        
        image_count += 1
        
    # Create metadata
    metadata = {
        "post_title": post_title,
        "post_slug": post_slug,
        "timestamp": timestamp,
        "image_count": image_count,
        "featured_images": os.path.relpath(featured_dir, ASSETS_DIR),
        "thumbnail_images": os.path.relpath(thumbnail_dir, ASSETS_DIR),
        "content_images": os.path.relpath(content_dir, ASSETS_DIR),
    }
    
    # Save metadata
    metadata_file = os.path.join(ASSETS_DIR, "deployment", "wordpress", f"{post_dir}-metadata.json")
    os.makedirs(os.path.dirname(metadata_file), exist_ok=True)
    with open(metadata_file, 'w') as f:
        json.dump(metadata, f, indent=2)
        
    return metadata

# Example usage (commented out)
# prepare_wordpress_assets("path/to/post/images", "My WordPress Post")

## HuggingFace Asset Preparation

Next, let's look at how we prepare assets for HuggingFace Spaces:

In [ ]:
def prepare_huggingface_assets(source_dir, space_name):
    """Prepare assets for a HuggingFace Space deployment"""
    space_slug = space_name.lower().replace(' ', '-')
    timestamp = datetime.now().strftime("%Y%m%d")
    space_dir = f"{timestamp}-{space_slug}"
    
    # Define output directories
    ui_dir = os.path.join(ASSETS_DIR, "optimized", "huggingface", "ui", space_dir)
    images_dir = os.path.join(ASSETS_DIR, "optimized", "huggingface", "images", space_dir)
    icons_dir = os.path.join(ASSETS_DIR, "optimized", "huggingface", "icons", space_dir)
    
    # Create directories
    os.makedirs(ui_dir, exist_ok=True)
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(icons_dir, exist_ok=True)
    
    # Process images based on subdirectory or naming convention
    image_count = 0
    source_path = Path(source_dir)
    
    # Function to determine asset type
    def determine_asset_type(img_path):
        path_str = str(img_path).lower()
        if "icon" in path_str or "logo" in path_str:
            return "icons"
        elif "ui" in path_str or "button" in path_str or "nav" in path_str:
            return "ui"
        else:
            return "images"
    
    for img_file in source_path.glob("**/*.{jpg,jpeg,png,gif}"):
        # Determine asset type
        asset_type = determine_asset_type(img_file)
        
        # File paths
        rel_path = img_file.relative_to(source_path)
        
        # Generate output path based on asset type
        if asset_type == "icons":
            output_dir = icons_dir
            config = platform_configs["huggingface"]["icons"]
        elif asset_type == "ui":
            output_dir = ui_dir
            config = platform_configs["huggingface"]["ui"]
        else:
            output_dir = images_dir
            config = platform_configs["huggingface"]["images"]
            
        # Ensure file extension matches format
        output_path = os.path.join(output_dir, f"{rel_path.stem}.{config['format'].lower()}")
        
        # Optimize the image
        resize_image(str(img_file), output_path, config)
        image_count += 1
        
    # Create deployment package
    deployment_dir = os.path.join(ASSETS_DIR, "deployment", "huggingface", space_dir)
    os.makedirs(deployment_dir, exist_ok=True)
    
    # Copy optimized assets to deployment directory
    shutil.copytree(ui_dir, os.path.join(deployment_dir, "ui"))
    shutil.copytree(images_dir, os.path.join(deployment_dir, "images"))
    shutil.copytree(icons_dir, os.path.join(deployment_dir, "icons"))
    
    # Create metadata
    metadata = {
        "space_name": space_name,
        "space_slug": space_slug,
        "timestamp": timestamp,
        "image_count": image_count,
        "total_size_kb": sum(os.path.getsize(os.path.join(dirpath, f)) 
                           for dirpath, dirnames, filenames in os.walk(deployment_dir) 
                           for f in filenames) // 1024,
        "asset_counts": {
            "ui": len(list(Path(ui_dir).glob("**/*.*"))),
            "images": len(list(Path(images_dir).glob("**/*.*"))),
            "icons": len(list(Path(icons_dir).glob("**/*.*"))),
        }
    }
    
    # Save metadata
    metadata_file = os.path.join(deployment_dir, "assets-metadata.json")
    with open(metadata_file, 'w') as f:
        json.dump(metadata, f, indent=2)
        
    return metadata

# Example usage (commented out)
# prepare_huggingface_assets("path/to/space/images", "My HF Space")

## Integration with WordPress and HuggingFace APIs

The asset organization system is designed to work with the WordPress and HuggingFace deployment scripts. Here's how you can use the optimized assets in your automated deployment pipelines:

In [ ]:
# WordPress API integration example
def upload_to_wordpress(post_dir):
    """Example function to upload assets to WordPress"""
    # This is a placeholder for the actual WordPress API integration
    # In the real implementation, this would use the WordPress REST API
    
    print(f"Loading assets from {post_dir}")
    print("Connecting to WordPress API...")
    print("Uploading assets...")
    print("Creating WordPress post with optimized assets...")
    
    # Example of how this would work with the WordPress API client
    """
    from wordpress_api_client import WordPressClient  # This is a fictional library
    
    wp = WordPressClient(site_url="https://yourblog.com", username="user", password="pass")
    
    # Upload featured image
    featured_path = os.path.join(ASSETS_DIR, "deployment", "wordpress", post_dir, "featured", "main-image.jpg")
    featured_id = wp.upload_media(featured_path)
    
    # Create post with the uploaded image
    post_data = {
        "title": "My Blog Post",
        "content": "<p>This is my post content with optimized images.</p>",
        "featured_media": featured_id,
        "status": "draft"
    }
    
    post_id = wp.create_post(post_data)
    print(f"Created post with ID: {post_id}")
    """
    
    return {"status": "success", "message": "Post assets uploaded to WordPress"}

# Example call (commented out)
# upload_to_wordpress("20230615-my-wordpress-post")

In [ ]:
# HuggingFace API integration example
def deploy_to_huggingface(space_dir):
    """Example function to deploy assets to HuggingFace"""
    # This is a placeholder for the actual HuggingFace API integration
    # In the real implementation, this would use the HuggingFace Hub API
    
    print(f"Loading deployment package from {space_dir}")
    print("Connecting to HuggingFace API...")
    print("Uploading assets to Space...")
    
    # Example of how this would work with the Hugging Face Hub client
    """
    from huggingface_hub import HfApi  # This is a real library
    
    api = HfApi(token="your_token")
    
    # Upload assets to the Space
    deployment_dir = os.path.join(ASSETS_DIR, "deployment", "huggingface", space_dir)
    
    # Upload each file in the deployment directory
    for root, dirs, files in os.walk(deployment_dir):
        for file in files:
            file_path = os.path.join(root, file)
            rel_path = os.path.relpath(file_path, deployment_dir)
            
            # Upload to Hugging Face Space
            api.upload_file(
                path_or_fileobj=file_path,
                path_in_repo=f"assets/{rel_path}",
                repo_id=f"your-username/your-space-name",
                repo_type="space"
            )
    """
    
    return {"status": "success", "message": "Assets deployed to HuggingFace Space"}

## Summary and Best Practices

The asset organization system provides a structured approach to managing assets across multiple deployment targets. Here are some best practices to follow:

1. **Always work with source assets** - Keep high-quality originals in the source directory
2. **Use the optimization scripts** - Don't manually resize or compress images
3. **Follow naming conventions** - Use descriptive names that indicate content and purpose
4. **Keep metadata current** - Update metadata files when assets change
5. **Use the deployment packages** - Let the system handle platform-specific optimizations

This notebook demonstrates the core functionality of the asset system. In production, these functions are implemented in the Python scripts located in the `assets/scripts` directory.